# Electrical Power Generation

In [1]:
%pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 11.4 MB/s eta 0:00:0000:010:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

## Input Data
We define all the input data of the model.

In [3]:
# Parameters

ntypes = 5 # added the new hydro plants
nperiods = 5 # added the new hydro plants
maxstart0 = 7 # changed to account for the new hydro plants (told by Mr. Parshan to assume they are available)

generators = [12, 10, 5, 1, 1] # one hydro plant each
period_hours = [6, 3, 6, 3, 6] # remain the same
demand = [15000, 30000, 25000, 40000, 27000] # stays the same
min_load = [850, 1250, 1500, 0, 0] # Since we will have to start the output from somewhere (0), it cannot start from a number 
max_load = [2000, 1750, 4000, 900, 1400] # 900 and 1400 for the fixed output of the hydro plants
base_cost = [1000, 2600, 3000, 90, 150] # 90 and 150 for the hydro plants 

per_mw_cost = [2, 1.3, 3, 0, 0] # nothing given for the new hydro plants, therefore 0 each

startup_cost = [2000, 1000, 500, 1500, 1200] # 1500 and 1200 for the hydro plants 

# added 0 each for the new hydro plants in the min_load since even though we know the output is fixed, but it would
# still have to start from some point (0) in order to go upto the fixed value which are 900 and 1400 respectively.

## Model Deployment

We create a model and the variables. For each time period, we have: an integer decision variable to capture the number of active generators of each type (ngen), an integer variable to capture the number of generators of each type we must start (nstart), and a continuous decision variable to capture the total power output for each generator type (output).

In [4]:
model = gp.Model('PowerGeneration')
ngen = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="ngen")
nstart = model.addVars(ntypes, nperiods, vtype=GRB.INTEGER, name="nstart")
output = model.addVars(ntypes, nperiods, vtype=GRB.CONTINUOUS, name="genoutput")

# a new height variable for the reservoir for each period
Height_p = model.addVar(nperiods, vtype = GRB.INTEGER, name = "height_p")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24


Next we insert the constraints:

The number of active generators can't exceed the number of generators.

In [5]:
# Generator count

numgen = model.addConstrs(ngen[type, period] <= generators[type]
                         for type in range(ntypes) for period in range(nperiods))

Total power output for a generator type depends on the number of generators of that type that are active.

In [6]:
# Respect minimum and maximum output per generator type

min_output = model.addConstrs((output[type, period] >= min_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

max_output = model.addConstrs((output[type, period] <= max_load[type] * ngen[type, period])
                              for type in range(ntypes) for period in range(nperiods))

Total output for each time period must meet predicted demand.

In [7]:
# Meet demand

meet_demand = model.addConstrs(gp.quicksum(output[type, period] for type in range(ntypes)) >= demand[period]
                               for period in range(nperiods))

Selected generators must be able to cope with an excess of demand.

In [8]:
# Provide sufficient reserve capacity

reserve = model.addConstrs(gp.quicksum(max_load[type]*ngen[type, period] for type in range(ntypes)) >= 1.15*demand[period]
                    for period in range(nperiods))

Connect the decision variables that capture active generators with the decision variables that count startups.

In [9]:
# Startup constraint

startup0 = model.addConstrs((ngen[type,0] <= maxstart0 + nstart[type,0])
                            for type in range(ntypes))

startup = model.addConstrs((ngen[type,period] <= ngen[type,period-1] + nstart[type,period])
                           for type in range(ntypes) for period in range(1,nperiods))

In [ ]:
# reservoir constraint

# reservoir = model.addConstr(Height_p = )

# For the reservoir, we need to introduce a new height variable for the reservoir for each period, add constraints that
# relate the heights across periods based on the amount of water pumped, and add another constraint linking the height
# at the end of the day to the beginning of the day.
# For example:
# let x_t be the level of water in meters in the reservoir
# x_1 = x_0 - [ 0.31 * (p_1) * (ngen_{A}) - 0.47 * (p_1) * (ngen_{B}) ]
# x_2 = .......
# .
# .
# .
# and in the end,
# x_t = x_0

Objective: minimize total cost.  Cost consists of three components: the cost for running active generation units, the cost to generate power beyond the minimum for each unit, and the cost to start up generation units.

In [10]:
# Objective: minimize total cost

active = gp.quicksum(base_cost[type]*period_hours[period]*ngen[type,period]
                    for type in range(ntypes) for period in range(nperiods))

per_mw = gp.quicksum(per_mw_cost[type]*period_hours[period]*(output[type,period] - min_load[type]*ngen[type,period])
                       for type in range(ntypes) for period in range(nperiods))

startup_obj = gp.quicksum(startup_cost[type]*nstart[type,period]
                         for type in range(ntypes) for period in range(nperiods))

model.setObjective(active + per_mw + startup_obj) # + reservoir constraint

Next, we start the optimization and Gurobi finds the optimal solution.

In [11]:
model.write('junk.lp')
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 110 rows, 76 columns and 235 nonzeros
Model fingerprint: 0xc4e54c7a
Variable types: 25 continuous, 51 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [5e+00, 5e+00]
  RHS range        [1e+00, 5e+04]
Presolve removed 48 rows and 14 columns
Presolve time: 0.01s
Presolved: 62 rows, 62 columns, 174 nonzeros
Variable types: 0 continuous, 62 integer (18 binary)
Found heuristic solution: objective 1089373.3000
Found heuristic solution: objective 990260.00000

Root relaxation: objective 8.984100e+05, 22 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 898410.000    0    9 9

---
## Analysis

The anticipated demand for electricity over the 24-hour time window can be met for a total cost of $\$1,002,540$. The detailed plan for each time period is as follows.

### Unit Commitments

The following table shows the number of generators of each type that are active in each time period in the optimal solution:

In [12]:
rows = ["Type" + str(t) for t in range(ntypes)]
units = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        units.loc["Type"+str(t), p] = ngen[t,p].x
units

,0,1,2,3,4
Type0,12.0,12.0,12.0,12.0,12.0
Type1,1.0,7.0,7.0,9.0,8.0
Type2,0.0,0.0,0.0,1.0,0.0
Type3,1.0,1.0,1.0,1.0,1.0
Type4,1.0,1.0,1.0,1.0,1.0


The following shows the number of generators of each type that must be started in each time period to achieve this plan (recall that the model assumes that 5 are available at the beginning of the time horizon):

In [13]:
startups = pd.DataFrame(columns=range(nperiods), index=rows, data=0.0)

for t in range(ntypes):
    for p in range(nperiods):
        startups.loc["Type"+str(t), p] = int(nstart[t,p].x)
startups

,0,1,2,3,4
Type0,5.0,0.0,0.0,0.0,0.0
Type1,0.0,6.0,0.0,2.0,0.0
Type2,0.0,0.0,0.0,1.0,0.0
Type3,0.0,0.0,0.0,0.0,0.0
Type4,0.0,0.0,0.0,0.0,0.0


In [14]:
model.SolCount
model.Params.SolutionNumber=0
model.getAttr(GRB.Attr.Xn)
model.getAttr(GRB.Attr.PoolObjVal)

900260.0